In [2]:
import numpy as np
import gc
from multiprocessing import Process
from tqdm import tqdm

def compare_files(file1, file2, num_keys_to_check=10, num_values_to_check=10):
    dict1 = np.load(file1, allow_pickle=True).item()
    keys1 = list(dict1.keys())
    dict1_small = {k: dict1[k].copy() for k in keys1[:num_keys_to_check]}
    del dict1
    gc.collect()

    dict2 = np.load(file2, allow_pickle=True).item()
    keys2 = list(dict2.keys())
    dict2_small = {k: dict2[k].copy() for k in keys2[:num_keys_to_check]}
    del dict2
    gc.collect()

    if keys1 != keys2:
        print("キーセットが一致しません。比較を中止します。")
    else:
        print("キーセットは一致しています。比較を続行します。")

        total_checked = 0
        total_different = 0
        differences = []

        for key in tqdm(dict1_small.keys(), desc="Comparing mel spectrograms"):
            arr1 = dict1_small[key]
            arr2 = dict2_small[key]

            if arr1.shape != arr2.shape:
                print(f"キー {key} の shape が違います: {arr1.shape} vs {arr2.shape}")
                continue

            flat1 = arr1.flatten()
            flat2 = arr2.flatten()
            n_to_check = min(num_values_to_check, len(flat1))

            for idx in range(n_to_check):
                diff = abs(flat1[idx] - flat2[idx])
                differences.append(diff)
                total_checked += 1
                if diff > 1e-5:
                    total_different += 1

        if total_checked > 0:
            print(f"チェックした総数: {total_checked}")
            print(f"違っていた数: {total_different}")
            print(f"違っていた割合: {100 * total_different / total_checked:.2f}%")
            print(f"平均絶対差 (MAE): {np.mean(differences):.8f}")
            print(f"最大差: {np.max(differences):.8f}")
        else:
            print("チェックできるデータがありませんでした。")

    del dict1_small
    del dict2_small
    gc.collect()

if __name__ == "__main__":
    file1 = '../data/processed/melspec_20250511_1805/birdclef2025_melspec_5sec_256_256.npy'
    file2 = '../data/processed/melspec_maxRMS/birdclef2025_melspec_5sec_256_256.npy'

    # サブプロセスで比較処理を実行
    p = Process(target=compare_files, args=(file1, file2))
    p.start()
    p.join()

    print("比較処理が終了し、メモリも完全に解放されました。")

キーセットは一致しています。比較を続行します。


Comparing mel spectrograms: 100%|██████████| 10/10 [00:00<00:00, 17218.00it/s]


チェックした総数: 100
違っていた数: 67
違っていた割合: 67.00%
平均絶対差 (MAE): 0.01782848
最大差: 0.12647223
比較処理が終了し、メモリも完全に解放されました。
